In [8]:
from google.cloud import bigquery
import pandas as pd
from IPython.display import display
import openpyxl
from openpyxl import Workbook

client = bigquery.Client()

# Run the original query and store the results in a dataframe
print("Running the query...")
query_original = """
SELECT fire_name, fire_year, address_input, address_city, address_state, address_zip, L1.value as ZFIRE_L1, L2.value AS ZFIRE_L2
FROM zestyai.atlas.wildland_fire_perimeters_all fires
JOIN zestyai-stark-prod.jobs.property_addresses PA on ST_WITHIN(PA.geocode_geo, fires.geog)
LEFT JOIN zestyai-stark-prod.jobs.property_labels L1
  ON PA.job_id = L1.job_id
  AND PA.property_id = L1.property_ID
  AND L1.feature = 'zfire-l1'
LEFT JOIN zestyai-stark-prod.jobs.property_labels L2
  ON PA.job_id = L2.job_id
  AND PA.property_id = L2.property_ID
  AND L2.feature = 'zfire-l2'
WHERE PA.job_id = 'pure_rfp_04042023@ep-zfire_2023-04-04'
"""
query_job_original = client.query(query_original)  # Make an API request.
df = query_job_original.to_dataframe()

# Set the datatype for the columns address_zip, ZFIRE_L1, and ZFIRE_L2 to numeric
df['address_zip'] = pd.to_numeric(df['address_zip'], errors='coerce')
df['ZFIRE_L1'] = pd.to_numeric(df['ZFIRE_L1'], errors='coerce')
df['ZFIRE_L2'] = pd.to_numeric(df['ZFIRE_L2'], errors='coerce')

# Print the original data
print("Fire Inclusion List:")
display(df)

# Print the distribution by fire name, state, and year
print("\nDistribution by Fire Name, State, and Year:")
fire_name_state_year_distribution = df.groupby(['fire_name', 'address_state', 'fire_year']).size().reset_index().rename(columns={0: 'Count'})
fire_name_state_year_distribution = fire_name_state_year_distribution.sort_values(by='Count', ascending=False)
display(fire_name_state_year_distribution)

# Create a new Excel workbook with two sheets
print("Creating Excel file...")
workbook = Workbook()
writer = pd.ExcelWriter("fire_data.xlsx", engine='openpyxl')
writer.book = workbook

# Remove the default sheet called "Sheet"
if 'Sheet' in writer.book:
    writer.book.remove(writer.book['Sheet'])

# Write the dataframes to the sheets
df.to_excel(writer, sheet_name="Fire Inclusion List", index=False)
fire_name_state_year_distribution.to_excel(writer, sheet_name="Distributions by Fire", index=False)

# Save the Excel file
writer.save()
print("Excel file 'fire_data.xlsx' with two sheets created.")

Running the query...
Fire Inclusion List:


,fire_name,fire_year,address_input,address_city,address_state,address_zip,ZFIRE_L1,ZFIRE_L2
0,SAWTOOTH COMPLEX,2006,"5476 Roy Rogers Rd, Pioneertown, CA 92268",Pioneertown,CA,92268,6,6.0
1,TOPANGA,2005,"186 Saddlebow Road, Bell Canyon, CA 91307",Bell Canyon,CA,91307,7,7.0
2,CREEK,2017,"12831 Maclay St, Sylmar, CA 91342",Los Angeles,CA,91342,3,4.0
3,HOLY,2018,"30356 Ainsworth Pl , Lake Elsinore, CA 92530",Lake Elsinore,CA,92530,6,3.0
4,GREEN KNOLL,2001,"2005 S Crescent H Ranch Road, Wilson, WY 83014",Wilson,WY,83014,6,NaN
...,...,...,...,...,...,...,...,...
805,TUBBS,2017,"4266 Wallace Rd , Santa Rosa, CA 95404",Santa Rosa,CA,95404,6,6.0
806,TUBBS,2017,"3704 Newbury Ct, Santa Rosa, CA 95404",Santa Rosa,CA,95404,6,8.0
807,TUBBS,2017,"7755 Foothill Ranch Rd , Santa Rosa, CA 95404",Santa Rosa,CA,95404,9,5.0
808,Marshall,2021,"303 W. Charles Street, Superior, CO 80027",Superior,CO,80027,4,5.0



Distribution by Fire Name:


,Fire Name,Count
0,WOOLSEY,168
1,TUBBS,165
2,Glass,92
3,THOMAS,84
4,KINCADE,43
...,...,...
75,PEDERNALES BEND,1
76,Lionshead,1
77,Alisal,1
78,Spring Creek,1


Creating Excel file...
Excel file 'fire_data.xlsx' with two sheets created.
